In [14]:
# imports
from skimage.filters import threshold_otsu, rank
from skimage.util import img_as_ubyte
from skimage.io import imread,imsave
import cv2 as cv
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import skimage as sk
from skimage.morphology import *
from helpers import *
from preprocessing import *
from extract_features import *
from classification import *
%matplotlib inline

In [15]:
# read image
# kufi_img = imread("./ACDB/ACdata_base/7/1128.jpg", as_gray=True)
# skufi_img = imread("./ACDB/ACdata_base/9/1550.jpg", as_gray=True)
# other_img = imread("./ACDB/ACdata_base/5/781.jpg", as_gray=True)

# img = get_skeleton(kufi_img)
# img2 = get_skeleton(skufi_img) 
# img3 = get_skeleton(other_img) 
# show_images([kufi_img,img])
# show_images([skufi_img,img2])
# show_images([other_img,img3])

# names = open("./ACDB/ACdata_base/names.txt","r")
# for i in range(1,10):
#     print('running LVL(0.1) on font {}\t\tfv:[ratio,var]'.format(names.readline()[:-1]))
#     imgsPaths = getImgsPaths("./ACDB/ACdata_base/"+str(i))
#     imgs = []
#     for i in range(len(imgsPaths)):
#         img = readImageGray(imgsPaths[i])
#         skeleton = get_skeleton(img)
#         imgs.append(skeleton)
#     mean,var = getDataAboutFeatures(imgs,getLVL)
#     print('(mean,var) for longest line norm={}'.format((mean[0],var[0])))
#     print('(mean,var) for ratio={}'.format((mean[1],var[1])))
#     print('(mean,var) for var among lines={}'.format((mean[2],var[2])))
#     print('....................................................')

    
    
    

In [8]:
appendCSV = False
for i in range(1, 10):
    i = 10-i
    print("\ndataset "+str(i))
    imgsPaths = getImgsPaths("./ACDB/ACdata_base/"+str(i))
    imgs = []
    for imgPath in imgsPaths:
        img = readImageGray(imgPath)
        img_skeleton = skeletonization(img)
        arr = getLVL(img_skeleton)
        saveArrayToCSV(arr, "csv/lvl.csv", str(i), appendCSV)
        appendCSV = True

appendCSV = False
for i in range(1, 10):
    print("\ndataset "+str(i))
    imgsPaths = getImgsPaths("./test_images/"+str(i))
    for imgPath in imgsPaths:
        img = readImageGray(imgPath)
        img_skeleton = skeletonization(img)
        arr = getLVL(img_skeleton)
        saveArrayToCSV(arr, "csv/lvl_test.csv", str(i), appendCSV)
        appendCSV = True



dataset 9

dataset 8

dataset 7

dataset 6

dataset 5

dataset 4

dataset 3

dataset 2

dataset 1

dataset 1

dataset 2

dataset 3

dataset 4

dataset 5

dataset 6

dataset 7

dataset 8

dataset 9


In [16]:

lvl, labels = readFromCSV("csv/lvl.csv") 
hpp, _ = readFromCSV("csv/hpp.csv")
lpq, _ = readFromCSV("csv/lpq.csv")
toe, _ = readFromCSV("csv/toe.csv")
tos, _ = readFromCSV("csv/tos.csv")
wor, _ = readFromCSV("csv/wor.csv")
hvsl, _ = readFromCSV("csv/hvsl.csv")

lvl_test, labels_test = readFromCSV("csv/lvl_test.csv") 
hpp_test, _ = readFromCSV("csv/hpp_test.csv")
lpq_test, _ = readFromCSV("csv/lpq_test.csv")
toe_test, _ = readFromCSV("csv/toe_test.csv")
tos_test, _ = readFromCSV("csv/tos_test.csv")
wor_test, _ = readFromCSV("csv/wor_test.csv")
hvsl_test, _ = readFromCSV("csv/hvsl_test.csv")

features_name = ['lpq', 'toe', 'tos', 'wor', 'hslv', 'lvl', 'hpp']
feature_values = [lpq, toe, tos, wor, hvsl, lvl, hpp]
feature_values_test = [lpq_test, toe_test, tos_test, wor_test, hvsl_test, lvl_test, hpp_test]
highest_acc = -1
f_used = None
for i in range(1, int(2**len(features_name))+1):
    features_used = []
    features_used_test = []
    features_used_indx = []
    bin_string = bin(i)[::-1]
    for i in range(len(bin_string)):
        if bin_string[i] == 'b':
            break
        if bin_string[i] == '1':
            indx = len(features_name)-i-1
            features_used.append(feature_values[indx])
            features_used_test.append(feature_values_test[indx])
            features_used_indx.append(indx)
    print("features used")
    for indx in features_used_indx:
        print(features_name[indx])
    if len(features_used) == 0:
        continue
    elif len(features_used) == 1:
        clf = svmClassifier(features_used[0], labels)
        #clf = adaboostClassifier(features_used[0], labels)
        # clf = randomForestClassifier(features_used[0], labels)
        acc = 0
        for i in range(features_used[0].shape[0]):
            f = np.reshape(features_used[0][i], (1, -1))
            if clf.predict(f) == labels[i]:
                acc += 1
        print("train accuracy ", 100*acc/len(labels))
        if 100*acc/len(labels) > highest_acc:
            highest_acc = 100*acc/len(labels)
            f_used = features_used_indx
        acc = 0
        for i in range(features_used_test[0].shape[0]):
            f = np.reshape(features_used_test[0][i], (1, -1))
            if clf.predict(f) == labels_test[i]:
                acc += 1
        print("test accuracy ", 100*acc/len(labels_test))

    else:
        allFeatures = np.concatenate(tuple(features_used), axis=1)
        allFeatures_test = np.concatenate(
            tuple(features_used_test), axis=1)

        clf = svmClassifier(allFeatures, labels)
        #clf = adaboostClassifier(allFeatures, labels)
        # clf = randomForestClassifier(allFeatures, labels)
        acc = 0
        for i in range(allFeatures.shape[0]):
            f = np.reshape(allFeatures[i], (1, -1))
            if clf.predict(f) == labels[i]:
                acc += 1
        print("train accuracy ", 100*acc/len(labels))
        if 100*acc/len(labels) > highest_acc:
            highest_acc = 100*acc/len(labels)
            f_used = features_used_indx
        acc = 0
        for i in range(allFeatures_test.shape[0]):
            f = np.reshape(allFeatures_test[i], (1, -1))
            if clf.predict(f) == labels_test[i]:
                acc += 1
        print("test accuracy ", 100*acc/len(labels_test))

print("\n\n\nhighest training acc  ", highest_acc)
print("when using")
for indx in f_used:
    print(features_name[indx])


features used
hpp
train accuracy  78.16023738872404
test accuracy  7.291666666666667
features used
lvl
train accuracy  34.480712166172104
test accuracy  18.75
features used
hpp
lvl
train accuracy  82.13649851632047
test accuracy  19.791666666666668
features used
hslv
train accuracy  41.839762611275965
test accuracy  10.416666666666666
features used
hpp
hslv
train accuracy  84.62908011869436
test accuracy  7.291666666666667
features used
lvl
hslv
train accuracy  53.768545994065285
test accuracy  15.625
features used
hpp
lvl
hslv
train accuracy  86.46884272997033
test accuracy  13.541666666666666
features used
wor
train accuracy  37.62611275964392
test accuracy  15.625
features used
hpp
wor
train accuracy  85.8753709198813
test accuracy  6.25
features used
lvl
wor
train accuracy  55.78635014836795
test accuracy  30.208333333333332
features used
hpp
lvl
wor
train accuracy  88.6053412462908
test accuracy  18.75
features used
hslv
wor
train accuracy  74.48071216617211
test accuracy  13.5416